In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import config

In [ ]:
#Load google.cloud.bigquery
%load_ext google.cloud.bigquery

In [ ]:
#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%%bigquery --use_rest_api ZRI_MF
SELECT *
FROM `high-empire-220313.ZRI.Multi_Family`

In [ ]:
year_columns = [x for x in ZRI_MF.columns if ('20' in x)]

In [ ]:
ZRI_MF_long = ZRI_MF.melt(id_vars = ['RegionID','RegionName','City','State','Metro','CountyName','SizeRank'],
            value_vars = year_columns).rename({'value':'ZRI','variable':'Date'}, axis = 1)

In [ ]:
ZRI_MF_long.loc[:,'Month'] = ZRI_MF_long['Date'].apply(lambda x: int(x[-2:]))
ZRI_MF_long.loc[:,'Year'] = ZRI_MF_long['Date'].apply(lambda x: int(x[1:5]))

All steps above to import the ZRI_MF column, reformat and create new columns

In [ ]:
# Import three tables with weather data from big query (zipcode level information)

In [ ]:
%%bigquery --use_rest_api high_temp
SELECT *
FROM `high-empire-220313.Weather.High_Temp_Merged`

In [ ]:
%%bigquery --use_rest_api low_temp
SELECT *
FROM `high-empire-220313.Weather.Low_Temp_Merged`

In [ ]:
%%bigquery --use_rest_api rain_fall
SELECT *
FROM `high-empire-220313.Weather.Rainfall_Merged`

In [ ]:
# Isolate the zip code from the GEOID for each table
high_temp.GEOID = high_temp['GEOID'].str[-5:].astype(int)
low_temp.GEOID = low_temp['GEOID'].str[-5:].astype(int)
rain_fall.GEOID = rain_fall['GEOID'].str[-5:].astype(int)

In [ ]:
# Correct spellings and typos in month labels
# Rename GEOID to RegionName to match ZRI Multi-Family Long form table (ZRI_MF_long)
high_temp = high_temp.iloc[:,2:-1].rename(columns = {'GEOID':'RegionName', 'October_':'October', 'Novemebr':'November'})
low_temp = low_temp.iloc[:,2:-1].rename(columns = {'GEOID':'RegionName', 'October_':'October', 'Novemebr':'November'})
rain_fall = rain_fall.iloc[:,2:-1].rename(columns = {'GEOID':'RegionName', 'October_':'October', 'Novemebr':'November'})

In [ ]:
# create list of month column names to be used in the pd.melt function
months = list(high_temp.columns)[1:-1]

In [ ]:
# Keep RegionName column as is
# Move month column names to values located in one column named 'Month'
# all weather values for related months in one column with name same as dataframe name
high_temp = high_temp.melt(id_vars = 'RegionName', value_vars = months ).rename({'value':'High_Temp','variable':'Month'}, axis = 1)
low_temp = low_temp.melt(id_vars = 'RegionName', value_vars = months ).rename({'value':'Low_Temp','variable':'Month'}, axis = 1)
rain_fall = rain_fall.melt(id_vars = 'RegionName', value_vars = months ).rename({'value':'Rain_Fall','variable':'Month'}, axis = 1)

In [ ]:
# create function to convert month names to month integer equivalent
def month_string_to_number(string):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
        'may':5,
        'jun':6,
        'jul':7,
        'aug':8,
        'sep':9,
        'oct':10,
        'nov':11,
        'dec':12
        }
    s = string.strip()[:3].lower()
    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

In [ ]:
# change all month names to integers using the above defined function
high_temp.Month = high_temp.Month.apply(month_string_to_number)
low_temp.Month = low_temp.Month.apply(month_string_to_number)
rain_fall.Month = rain_fall.Month.apply(month_string_to_number)

In [ ]:
weather = high_temp.merge(low_temp, how = 'outer', on = ['RegionName','Month'])
weather = weather.merge(rain_fall, how = 'outer', on = ['RegionName','Month'])

In [ ]:
from data_setup import FixID

In [ ]:
weather = weather.assign(ZipCode = weather.RegionName.apply(FixID)).drop('RegionName',axis = 1)

In [ ]:
weather.to_pickle('./pickles/weather.p', protocol = 4)

In [ ]:
pd.set_option('display.max_rows', 150)

In [ ]:
# Merge each weather dataframe with the ZRI_MF_long dataframe
ZRI_weather = ZRI_MF_long.merge(high_temp,how = 'left', on=['RegionName','Month'])
ZRI_weather = ZRI_weather.merge(low_temp,how = 'left', on=['RegionName','Month'])
ZRI_weather = ZRI_weather.merge(rain_fall,how = 'left', on=['RegionName','Month'])

In [ ]:
ZRI_weather.to_pickle('./pickles/ZRI_weather.p', protocol = 4)